# BRC case study

This case study concerns the transient difference absorption measured in the visible and near-infrared spectral region after 880 nm
excitation from purple bacterial reaction center (RC). The measurement, the global analysis and the target analysis is based upon the model explained in ([Zhu et al. 2013][Zhu2013]).

There are 4 exercise questions in this second notebook. The notebook can only be executed properly when the missing starting values for some of the parameters have been inserted in the relevant parameter files.

[Zhu2013]: https://www.nat.vu.nl/~ivo/pub/2013/JingyiBRC_BJ104_2493.pdf "Zhu J, van Stokkum Ivo HM, Paparelli L, Jones Michael R, Groot Marie L (2013) Early Bacteriopheophytin Reduction in Charge Separation in Reaction Centers of Rhodobacter sphaeroides. Biophysical Journal 104 (11):2493-2502."

### Inspect data

In [ ]:
from pyglotaran_extras import plot_data_overview

BRC_NIR_data_path = "BRC_Pexc_NIR.ascii"
plot_data_overview(BRC_NIR_data_path, nr_of_data_svd_vectors=2, linlog=True);

## Global Analysis: sequential scheme with decay rates in decreasing order

### Used model and parameters

In [ ]:
from glotaran.utils.ipython import display_file

BRC_NIR_global_model_path = "models/BRC_NIR-global-model.yml"
BRC_NIR_global_parameters_path = "models/BRC_NIR-global-parameters.yml"

#### Model file

In [ ]:
display_file(BRC_NIR_global_model_path, syntax="yml")

#### Parameters file

The free parameters are the center (the location of the maximum of the IRF at 1000 nm, the dispersion_center) and the width of the IRF, and the kinetic parameters. The first decay rate has been fixed at 31/ps, it describes ultrafast relaxation and the coherent artefact. The final state is long lived. The final decay rate has been fixed at a very small number to avoid numerical problems with the optimization.
The dispersion parameters will be estimated from the data.

Exercise 5: estimate the missing starting values for the parameters from the data inspection, and insert these in the models/BRC_NIR-global-parameters.yml file.

In [ ]:
display_file(BRC_NIR_global_parameters_path, syntax="yml")

### Create scheme and optimize it

In [ ]:
from glotaran.optimization.optimize import optimize
from glotaran.project.scheme import Scheme

BRC_NIR_global_scheme = Scheme(
    model=BRC_NIR_global_model_path,
    parameters=BRC_NIR_global_parameters_path,
    maximum_number_function_evaluations=7,
    data={"BRC_NIR_data": BRC_NIR_data_path},
)
BRC_NIR_global_scheme.validate()

In [ ]:
BRC_NIR_global_result = optimize(BRC_NIR_global_scheme)

In [ ]:
BRC_NIR_global_result

In [ ]:
BRC_NIR_global_result.optimized_parameters

### Result plots

#### Fit quality

In [ ]:
from pyglotaran_extras import plot_fitted_traces, select_plot_wavelengths

wavelengths = select_plot_wavelengths(BRC_NIR_global_result.data["BRC_NIR_data"])
plot_fitted_traces(BRC_NIR_global_result.data["BRC_NIR_data"], wavelengths);

In [ ]:
fig_global, axes = plot_fitted_traces(
    BRC_NIR_global_result.data["BRC_NIR_data"], wavelengths, linlog=True, linthresh=1
)

Note that in the second plot the time axis is linear until 1 ps and logarithmic thereafter. This plot is more informative.

#### Overview

In [ ]:
from pyglotaran_extras import plot_overview

plot_overview(
    BRC_NIR_global_result.data["BRC_NIR_data"],
    linlog=True,
    figure_only=False,
    nr_of_data_svd_vectors=4,
    nr_of_residual_svd_vectors=1,
);

Exercise 6: Comment on the interpretation of the EADS, with the help of literature. Which EADS is the most difficult to interpret?

## First Target Analysis: inverted kinetics

### Used model and parameters

In [ ]:
BRC_NIR_target_model_path = "models/BRC_NIR-target-model.yml"
BRC_NIR_target_parameters_path = "models/BRC_NIR-target-parameters.yml"

#### Model file

In [ ]:
display_file(BRC_NIR_target_model_path, syntax="yml")

#### Parameters file

Exercise 7: In the regular sequential scheme (with decay rates in decreasing order) you can interchange two of the decay rates to model the inverted kinetics that the second electron transfer step is faster than the first.
Insert the missing starting values in the models/BRC_NIR-target-parameters.yml file. Hint: use the above estimated kinetic and IRF parameters.

In [ ]:
display_file(BRC_NIR_target_parameters_path, syntax="yml")

The free parameters are the center (the location of the maximum of the IRF) and the width of the IRF, and some of the kinetic parameters.
Which kinetic parameters can be estimated is difficult, and requires some trial and error. Six eigenvalues (the reciprocals of the lifetimes) of the K matrix can in principle be estimated and spectral assumptions are needed for the remaining kinetic parameters. Because of the limited time range measured(less than 100 ps), it is better to fix the Chl decay rate at 1/(2ns).

### Create scheme and optimize it

In [ ]:
BRC_NIR_target_scheme = Scheme(
    model=BRC_NIR_target_model_path,
    parameters=BRC_NIR_target_parameters_path,
    maximum_number_function_evaluations=7,
    data={"BRC_NIR_data": BRC_NIR_data_path},
)
BRC_NIR_target_scheme.validate()

In [ ]:
BRC_NIR_target_result = optimize(BRC_NIR_target_scheme)

In [ ]:
BRC_NIR_target_result

### Result plots

#### Fit quality

In [ ]:
fig_target, axes = plot_fitted_traces(
    BRC_NIR_target_result.data["BRC_NIR_data"], wavelengths, linlog=True, linthresh=1
);

#### Overview

In [ ]:
plot_overview(
    BRC_NIR_target_result.data["BRC_NIR_data"],
    figure_only=False,
    nr_of_data_svd_vectors=3,
    nr_of_residual_svd_vectors=1,
    linlog=True,
    linthresh=1,
);

Exercise 8: Comment on the interpretation of the SADS, with the help of literature. Which SADS is the most difficult to interpret?

In [ ]:
BRC_NIR_target_result.optimized_parameters

In [ ]:
compartments = BRC_NIR_target_scheme.model.initial_concentration["input1"].compartments

BRC_NIR_target_scheme.model.k_matrix["km1"].matrix_as_markdown(compartments).replace(
    "0.0000e+00", ""
)